# Federated ML
## Contents
1. Introduction
1. Reading an image from the dataset
1. Loading Images and Labels
1. Splitting training and testing data
1. Creating clients
1. Creating helper nodes
1. Batching clients' and test data
1. Creating an MLP model
1. Optimizer, Loss function and Metrics to compile the model
1. Utility functions for the Federated Averaging
1. Functions to fetch and set shape of model weights
1. Modulation and Demodulation functions for Communication
1. Transmission functions from Client to Helper and Helper to Master
1. Model test function
1. Federated Averaging Algorithm

## Introduction
In this notebook, we have implemented a working Federated ML model. The goal is to simulate packet losses in communication between the server and the clients in the Federated Averaging Algorithm. The function once implemented will be added to the algorithm near the TODO's in the code.

Reading an image from the dataset

In [ ]:
import cv2
import matplotlib.pyplot as plt
image = cv2.imread("datasets/numbers/trainingSet/0/img_1.jpg")
plt.imshow(image)
plt.title("Image")

Loading Images and Labels

In [ ]:
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
dir_path = "datasets/numbers/trainingSet/"
images = list()
labels = list()
for number in range(0, 10):
    folder = dir_path + str(number)
    for image_file in os.listdir(folder):
        image_gray = cv2.imread(os.path.join(folder, image_file), cv2.IMREAD_GRAYSCALE)
        image = np.array(image_gray).flatten()
        images.append(image/255)
        labels.append(number)
    print("[INFO] Processed {}/{}".format(number + 1, 10))
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

Splitting training and testing data

In [ ]:
from sklearn.model_selection import train_test_split
images_train, images_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.1, random_state=42)

Creating clients

In [ ]:
import random
num_clients = 16
client_names = ["client_{}".format(i + 1) for i in range(num_clients)]
data = list(zip(images, labels))
random.shuffle(data)
size = len(data)//num_clients
data_shards = [data[i:i + size] for i in range(0, size*num_clients, size)]
clients = {client_names[i]: data_shards[i] for i in range(num_clients)}

Creating helper nodes

In [ ]:
import math
num_helpers = math.ceil(math.log2(num_clients))

Batching clients' and test data

In [ ]:
import tensorflow as tf
batch_size = 32
clients_batched = dict()
for (client_name, data_shard) in clients.items():
    data, labels = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(labels)))
    clients_batched[client_name] = dataset.shuffle(len(labels)).batch(batch_size)
test_batched = tf.data.Dataset.from_tensor_slices((images_test, labels_test)).batch(len(labels_test))

Creating the Multi Layer Perception (MLP) model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

Optimizer, Loss function and Metrics to compile the model

In [ ]:
from keras.optimizers import SGD
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, decay=lr/comms_round, momentum=0.9)

Utility functions for the Federated Averaging

In [ ]:
def weight_scaling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count

def scale_model_weights(weight, scalar):
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    avg_grad = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad

Functions to fetch and set shape of model weights

In [ ]:
def get_shape(weights):
    shapes = [i.shape for i in weights]
    return shapes

def flatten_model_weights(weights):
    flattened_weights = np.concatenate([i.flatten() for i in weights])
    return flattened_weights

def restore_model_shape(flattened_weights, shapes):
    weights = []
    index = 0
    for shape in shapes:
        size = np.product(shape)
        arr = np.array(flattened_weights[index : index + size])
        weights.append(arr.reshape(shape))
        index += size
    return weights

Modulation and Demodulation Functions for Communication

In [ ]:
from bitstring import BitArray

def modulation(weights):
    weights = flatten_model_weights(weights)
    packets = []
    for i in range(len(weights)):
        packet = BitArray(float=weights[i], length=32)
        packets.append(packet.bin)
    return packets

def demodulation(weights, model_shape):
    return restore_model_shape(weights, model_shape)

Transmission functions from Client to Helper and Helper to Master

In [ ]:
import random

def calculateDecodingProbability(n, q):
    if n < 3:
        return 0
    k = 3
    num, den = 1, 1
    for i in range(0, k):
        num *= (1 - q**(i - n))
        den *= (1 - q**(i - n + 1))
    return 1 - (1 - num)/(1 - den)

def transmissionCH(packet_loss_prob, q):
    succesfully_transmitted = 0
    transmissions = []
    transmissionCnt = 0
    for i in range(4):
        transmissionCnt += 1
        if random.random() > packet_loss_prob:
            transmissions.append(1)
            succesfully_transmitted += 1
            probability_success = calculateDecodingProbability(succesfully_transmitted, q)
            if random.random() < probability_success:
                return transmissions, True
        else:
            transmissions.append(0)
    return transmissions, False

Model test function

In [ ]:
from keras.losses import CategoricalCrossentropy
from sklearn.metrics import accuracy_score
def test_model(X_test, Y_test,  model, comm_round):
    cce = CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round+1, acc, loss))
    return acc, loss

Federated Averaging Algorithm

In [ ]:
from keras import backend as K

def fedML(packet_loss_prob=0, q=2**8):
    print(q)
    smlp_global = SimpleMLP()
    global_model = smlp_global.build(784, 10)
    model_shape = get_shape(global_model.get_weights())
    accuracy = []
    for comm_round in range(comms_round):
        global_weights = global_model.get_weights()
        client_names= list(clients_batched.keys())
        scaled_weights_list = []
        comm_matrix = []
        random.shuffle(client_names)
        lost_clients = 0
        for client_number, client in enumerate(client_names):
            smlp_local = SimpleMLP()
            local_model = smlp_local.build(784, 10)
            local_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
            local_model.set_weights(global_weights)
            local_model.fit(clients_batched[client], epochs=1, verbose=0)
            scaling_factor = weight_scaling_factor(clients_batched, client)
            scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
            bitWeights = modulation(scaled_weights)
            transmissions, decodable = transmissionCH(packet_loss_prob, q)
            comm_matrix.append(transmissions)
            if not decodable:
                lost_clients += 1
            else:
                weights = bitWeights
                for i in range(len(bitWeights)):
                    weights[i] = BitArray(bin=bitWeights[i]).float
                scaled_weights_list.append(weights)
            K.clear_session()
        if lost_clients != 0:
            compensated_lost_weights = scale_model_weights(global_weights, lost_clients*weight_scaling_factor(clients_batched, 'client_1'))
            packets = modulation(compensated_lost_weights)
            weights = packets
            for i in range(len(packets)):
                weights[i] = BitArray(bin=packets[i]).float
            scaled_weights_list.append(weights)
        global_weights = sum_scaled_weights(scaled_weights_list)
        global_weights = demodulation(global_weights, model_shape)
        global_model.set_weights(global_weights)
        with open('output/rlnc/Q{}/comm_matrix{}.txt'.format(q, 100*packet_loss_prob), 'a') as f:
            f.write('Round ' + str(comm_round) + ': ' + str(comm_matrix) + '\n')
        for(X_test, Y_test) in test_batched:
            global_acc, _ = test_model(X_test, Y_test, global_model, comm_round)
            accuracy.append(global_acc)
    return accuracy

In [ ]:
acc = fedML(0.25, 4)
np.savetxt('output/rlnc/Q{}/comm_matrix{}.txt'.format(4, 25), acc, delimiter=",")

In [ ]:
acc = fedML(0.25, 8)
np.savetxt('output/rlnc/Q{}/comm_matrix{}.txt'.format(8, 25), acc, delimiter=",")

In [ ]:
acc = fedML(0.25, 16)
np.savetxt('output/rlnc/Q{}/comm_matrix{}.txt'.format(16, 25), acc, delimiter=",")

In [ ]:
acc = fedML(0.25, 64)
np.savetxt('output/rlnc/Q{}/comm_matrix{}.txt'.format(64, 25), acc, delimiter=",")

In [ ]:
acc = fedML(0.25, 256)
np.savetxt('output/rlnc/Q{}/comm_matrix{}.txt'.format(256, 25), acc, delimiter=",")